In [5]:
from bs4 import BeautifulSoup
import requests
# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import math
import mysql.connector
from datetime import date
import json

In [6]:
import selenium

# Check version I am running
selenium.__version__

'4.15.2'

In [7]:
import undetected_chromedriver as uc

In [8]:
from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

In [9]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [10]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/browser-headers',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
header_browser_list = response.json()
print(header_browser_list['result'][0])

{'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3', 'sec-ch-ua': 'Google Chrome;v="84", "Chromium";v="84", ";Not A Brand";v="99"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': 'macOS', 'sec-fetch-site': 'none', 'sec-fetch-mod': '', 'sec-fetch-user': '?1', 'accept-encoding': 'gzip, deflate', 'accept-language': 'en-US,en;q=0.5'}


In [11]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/user-agents',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
user_agent_list = response.json()
print(user_agent_list['result'][0])

Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.106 Safari/537.36


In [12]:
random_index_user_agent = randint(0, len(user_agent_list)-1)
random_index_header_browser = randint(0, len(header_browser_list)-1)
user_agent_random = user_agent_list['result'][random_index_user_agent]
header_browser_random = header_browser_list['result'][random_index_header_browser]

In [26]:
def create_option():
    random_index_user_agent = randint(0, len(user_agent_list)-1)
    random_index_header_browser = randint(0, len(header_browser_list)-1)
    user_agent_random = user_agent_list['result'][random_index_user_agent]
    header_browser_random = header_browser_list['result'][random_index_header_browser]
    # Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
    option = uc.ChromeOptions()

    # Going undercover:
    option.add_argument("--incognito")


    # # Consider this if the application works and you know how it works for speed ups and rendering!

    # option.add_argument('--headless=chrome')
    user_agent = user_agent_random
    option.add_argument(f"user-agent={user_agent}")

    # Thêm header vào Options
    headers = header_browser_random

    for key, value in headers.items():
        option.add_argument(f"--header={key}:{value}")
    
    return option

In [27]:
option = create_option()
driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)
driver.implicitly_wait(10)  # Set the implicit wait time to 10 seconds
driver.set_page_load_timeout(30)
driver.get("https://www.topcv.vn/viec-lam-it")

job_count = driver.find_element(By.CSS_SELECTOR,'[class="job-header"] h2 b').text.replace(",", "")
if (int(job_count) % 50 == 0):
    max_page  = int(job_count) / 50
else:
    max_page = math.floor(int(job_count) / 50) + 1
print(max_page)
driver.quit()

53


In [29]:
job_urls = []

In [28]:
driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=create_option())
for page_number in range(1, int(max_page+1)):
    link = f"https://www.topcv.vn/viec-lam-it?page={page_number}"
    driver.get(link)
    print("Đang xử lý: ", link)
    job_url_list = driver.find_elements(By.CSS_SELECTOR, '.job-item-2 h3[class="title"] a')
    print(job_url_list)
    print("******************************************************************")
    for uu in job_url_list:
        link = uu.get_attribute("href")
        job_urls.append(link)
driver.quit()

Đang xử lý:  https://www.topcv.vn/viec-lam-it?page=1
[<undetected_chromedriver.webelement.WebElement (session="f3f12bfb54a3194246087d9b3e929e0a", element="f.A700C5015FEC7A8B14142944B9327D0F.d.34888DAC4DFE8B8D80E116B3381393AF.e.371")>, <undetected_chromedriver.webelement.WebElement (session="f3f12bfb54a3194246087d9b3e929e0a", element="f.A700C5015FEC7A8B14142944B9327D0F.d.34888DAC4DFE8B8D80E116B3381393AF.e.372")>, <undetected_chromedriver.webelement.WebElement (session="f3f12bfb54a3194246087d9b3e929e0a", element="f.A700C5015FEC7A8B14142944B9327D0F.d.34888DAC4DFE8B8D80E116B3381393AF.e.373")>, <undetected_chromedriver.webelement.WebElement (session="f3f12bfb54a3194246087d9b3e929e0a", element="f.A700C5015FEC7A8B14142944B9327D0F.d.34888DAC4DFE8B8D80E116B3381393AF.e.374")>, <undetected_chromedriver.webelement.WebElement (session="f3f12bfb54a3194246087d9b3e929e0a", element="f.A700C5015FEC7A8B14142944B9327D0F.d.34888DAC4DFE8B8D80E116B3381393AF.e.375")>, <undetected_chromedriver.webelement.WebEl

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.70)
Stacktrace:
	GetHandleVerifier [0x00978CD3+51395]
	(No symbol) [0x008E5F31]
	(No symbol) [0x0079E13A]
	(No symbol) [0x007825F2]
	(No symbol) [0x007F9B2B]
	(No symbol) [0x00809E66]
	(No symbol) [0x007F3C26]
	(No symbol) [0x007CC629]
	(No symbol) [0x007CD40D]
	GetHandleVerifier [0x00CF6493+3711107]
	GetHandleVerifier [0x00D3587A+3970154]
	GetHandleVerifier [0x00D30B68+3950424]
	GetHandleVerifier [0x00A29CD9+776393]
	(No symbol) [0x008F1704]
	(No symbol) [0x008EC5E8]
	(No symbol) [0x008EC799]
	(No symbol) [0x008DDDC0]
	BaseThreadInitThunk [0x767B7BA9+25]
	RtlInitializeExceptionChain [0x7796BD2B+107]
	RtlClearBits [0x7796BCAF+191]


In [11]:
conn = mysql.connector.connect(
    host='103.56.158.31',
    port = 3306,
    user= 'tuyendungUser',
    password='sinhvienBK',
    database= 'ThongTinTuyenDung'
)
cursor = conn.cursor()

In [12]:
sql = 'INSERT IGNORE INTO Stg_ThongTin_raw(Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong) VALUES (%s, %s, %s, %s,%s,%s,%s, %s, %s, %s, %s, %s, %s, %s,%s)'

In [13]:
sql_link = 'SELECT Link FROM Stg_ThongTin_raw where Web =\'TopCV\''

In [14]:
cursor.execute(sql_link)
result = cursor.fetchall()
remove_url_list = [row[0] for row in result]

In [15]:
print("Số lượng url cào về: ", len(job_urls))
for job_url in remove_url_list:
    if job_url in job_urls:
        job_urls.remove(job_url)
print("Số lượng url mới lấy được sau xử lý:", len(job_urls))

Số lượng url cào về:  50
Số lượng url mới lấy được sau xử lý: 50


In [16]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=create_option())
if len(job_urls) > 0:
    for job_url in job_urls:
        if "https://www.topcv.vn/brand/" in job_url:
            continue
        link = job_url
        print("Đang xử lí....", link)
        driver.get(link)
        Web = 'TopCV'
        Nganh = 'IT'
        Link = link
        TenCV = driver.find_element(By.CSS_SELECTOR, '.job-detail__info--title').text
        CongTy = driver.find_element(By.CSS_SELECTOR, '[class="company-name-label"] a').text
        SoLuong = "1"
        YeuCau = ""
        MoTa = ""
        PhucLoi = ""
        for i in range(len(driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]'))):
            if "Mức lương" in driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-title"]').text:
                Luong = driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-value"]').text
            if "Địa điểm" in driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-title"]').text:
                TinhThanh = driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-value"]').text
        try:
            HanNopCV = driver.find_element(By.CSS_SELECTOR, '[class="job-detail__info--deadline"]').text.split(":")[-1].strip()
        except:
            HanNopCV = date.today()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        List = soup.find_all('div', class_='job-description__item')
        YeuCau = List[1].text
        MoTa = List[0].text
        PhucLoi = List[2].text
        for i in range(len(soup.find_all('div', class_='box-general-group'))):
            if "Số lượng tuyển" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                SoLuong = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text.split()[0]
            if "Hình thức làm việc" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                LoaiHinh = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
            try:
                if "Kinh nghiệm" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                    KinhNghiem = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
            except:
                KinhNghiem = "Không có"
            if "Cấp bậc" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                CapBac = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
        cursor.execute(sql, (Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong))
        conn.commit()
else:
    print("Không có tin mới để thêm.")
driver.quit()
cursor.close()
conn.close()


KeyboardInterrupt: 

In [ ]:
print(job_urls)

['https://www.topcv.vn/viec-lam/ky-su-cau-noi-tieng-han-bien-phien-dich-tieng-han-nganh-it-korean-brse/1132735.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/vfx-artist-junior/1168046.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/lap-trinh-vien-software-developers-muc-luong-den-35m/1154432.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/nhan-vien-ky-thuat-may-tinh/1206111.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/brand/tek-experts/tuyen-dung/it-support-engineers-microsoft-software-support-fresher-experienced-j654470.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/tester-mobile-app/1016516.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/chuyen-vien-kiem-thu-phan-mem-mang-chung-khoan/1206536.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/senior-database-administrator-dba/1206364.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/android-java-develo